In [1]:
import tensorflow as tf
import os
import numpy as np

print(tf.__version__)

2.0.0


#### 周杰伦歌词数据:
链接: https://pan.baidu.com/s/1QieFe3iuDlDeyTYe4dRySA 提取码: vwvs

In [2]:
#读取数据
import random
import zipfile

with zipfile.ZipFile('./data/jaychou_lyrics.txt.zip') as zin:
    with zin.open('jaychou_lyrics.txt') as f:
        corpus_chars = f.read().decode('utf-8')

In [3]:
corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
corpus_chars = corpus_chars[0:10000]

In [4]:
# 建立字符索引
idx_to_char = list(set(corpus_chars))
char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
vocab_size = len(char_to_idx)

In [5]:
corpus_indices = [char_to_idx[char] for char in corpus_chars]
sample = corpus_indices[:20]
print('chars:', ''.join([idx_to_char[idx] for idx in sample]))
print('indices:', sample)

chars: 想要有直升机 想要和你飞到宇宙去 想要和
indices: [843, 28, 744, 39, 314, 321, 965, 843, 28, 1005, 617, 878, 317, 297, 360, 406, 965, 843, 28, 1005]


In [6]:
# one-hot向量
tf.one_hot(np.array([0, 2]), vocab_size)


<tf.Tensor: id=4, shape=(2, 1027), dtype=float32, numpy=
array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)>

In [7]:
def to_onehot(X, size): 
    # X shape: (batch), output shape: (batch, n_class)
    return [tf.one_hot(x, size,dtype=tf.float32) for x in X.T]
X = np.arange(10).reshape((2, 5))
inputs = to_onehot(X, vocab_size)
len(inputs), inputs[0].shape

(5, TensorShape([2, 1027]))

In [28]:
a = tf.reshape(tf.constant([1,2,3,4,5,6,7,8]), (2,4))
b = tf.reshape(tf.constant([1,2,3,4]),(1,4))
print(a + b)
print(a)
print(tf.zeros(shape=(1,5)))
print(tf.zeros(5))
print(tf.zeros(shape=(5,1)))

tf.Tensor(
[[ 2  4  6  8]
 [ 6  8 10 12]], shape=(2, 4), dtype=int32)
tf.Tensor(
[[1 2 3 4]
 [5 6 7 8]], shape=(2, 4), dtype=int32)
tf.Tensor([[0. 0. 0. 0. 0.]], shape=(1, 5), dtype=float32)
tf.Tensor([0. 0. 0. 0. 0.], shape=(5,), dtype=float32)
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]], shape=(5, 1), dtype=float32)


In [ ]:
# 从零实现 rnn
# 初始化模型参数，我们依据第二章的结论，使用 Ht = theta(concate(Xt + Ht-1, axis=-1) * W),所以可训练的模型参数是 W，输出矩阵Wo

# 初始化参数有：Ht-1 的维度
vocab_size = 1027
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
class Rnn(object):
    def __init__(self, hiden_dim):
        """
        初始化待训练的参数 variabel
        """
        self.hidden_weight = self._ones(shape=(vocab_size+ num_hiddens, num_hiddens))
        self.weight_out = self._ones(shape=(num_hiddens, vocab_size + num_hiddens))
        self.bn_h = tf.Variable(tf.zeros(1,num_hiddens))
        self,bn_o = tf.Variable(tf.zeros(1, vocab_size + num_hiddens))
        
    def _ones(self,shape):
        return tf.Variable(tf.random.normal(shape=shape,stddev=0.01,mean=0,dtype=tf.float32))
    
    def net(self,x):
        pass
